In [1]:
"""
v1.2

3/29/21

inspiration: https://www.youtube.com/watch?v=uWCqlo893tk

add stocks to the list that arent included like LOW

inflation target = 2%
"""

'\nv1.1\n\n3/29/21\n\nAdd highest year dividend growth rate (10 y)\nAdd Dividend growth rate\ninspiration: https://www.youtube.com/watch?v=uWCqlo893tk\n\nadd stocks to the list that arent included like LOW\n\ninflation target = 2%\n'

In [2]:
import pandas as pd
import numpy as np
import import_ipynb
import dividend_detective_scrape

import yfinance as yf

import datetime as dt

from datetime import datetime

from numpy import mean

importing Jupyter notebook from dividend_detective_scrape.ipynb


In [3]:
def trim_div_data(stockObj):
    actns = stockObj.actions
    for i,r in actns.iterrows():
        if r['Dividends'] > 0:
            first_div_date = i
            break

    actns = actns.loc[first_div_date:]
    return actns, first_div_date

In [4]:
def add_div_data_details(actns, first_div_date):
    actns['days_since_first_div'] = (actns.index-first_div_date).days
    actns['year'] = actns.index.year
    
    h = []
    l = []
    for i,r in actns.iterrows():
        prices = stockObj.history(start=i,end = i+pd.DateOffset(1)).iloc[0]
        h.append(prices['High'])
        l.append(prices['Low'])

    actns['Stock_High'] = h
    actns['Stock_Low'] = l
    
    actns['day_div_yield'] = actns['Dividends']/actns['Stock_High']
    return actns

In [5]:
def get_ind_div_data(stockObj):
    df, first_div_date = trim_div_data(stockObj)
    return add_div_data_details(df, first_div_date)

In [6]:
"""

then learn how long it will take to beat other investments...

"""

'\nwork something below to get div growth rate\n\nparse out below to get the different functions of the below section so I can work on it\n\nthen learn how long it will take to beat other investments...\n\n'

In [7]:
def get_historical_div_info(stockObj):
    df = stockObj.actions
    df['year'] = df.index.year
    df = df[df['Dividends']>0]
    df = df.drop(columns=['Stock Splits'])
    return df

In [8]:
def group_divs_by_year(df):
#     combine same year divs and rename column
    ann_df = df.groupby(['year']).agg(
        {'Dividends': np.sum}).rename(columns=
        {'Dividends':'dividends'})
    start_year = ann_df.index[0]
    end_year = ann_df.index[len(ann_df)-1]+1
    ann_df = ann_df.reindex(list(range(start_year,end_year)), fill_value=0)
    return ann_df

In [9]:
def add_consistancy_coeff(df):
    consistancy_coeff = [1/(2**(x+1)) for x in range(len(df))][::-1]
    year_progress = datetime.now().timetuple().tm_yday/365
    consistancy_coeff[-1] = consistancy_coeff[-1]*year_progress
    
    df['div_consistency_score'] = consistancy_coeff
    return df

In [10]:
def calculate_div_consistency_score(df):
    
    df = add_consistancy_coeff(df)

    # default highest div is the first div
    highest_div = df.iloc[0]['dividends']
    
#     add another weight to the last cell where i get the percentage of the year we've been in and multiply it by that
# exclude 

    for i,r in df[1:-1].iterrows():      
#         i need to think about a better way to do this because right now
#         it compares the wrong two pieces
        if r['dividends'] < df.loc[i-1]['dividends']:
            mult = r['div_consistency_score']
            div_perc = r['dividends']/highest_div
            df.at[i, 'div_consistency_score'] = mult * div_perc
        else:
            highest_div = r['dividends']
    
    return df['div_consistency_score'].sum()

In [11]:
def structure_df(df, years):
    df = df[:-years-3:-1]
    df = df[1:]
    return df

In [12]:
def get_div_deltas(df):
    
    div_deltas = []
        
    for i,r in df[:-1].iterrows():
        c = float(df.loc[i]['dividends'])
        p = float(df.loc[i-1]['dividends'])

        div_deltas.append((c-p)/p)
    
    return div_deltas

In [13]:
def get_div_growth_rate(df, years):
    if years >= len(df)-1:
        return 0
    else:
        df = structure_df(df, years)
        
        div_deltas = get_div_deltas(df)
        
    return mean(div_deltas)
            

In [14]:
def get_div_analysis(stockObj):

    df = get_historical_div_info(stockObj)
    ann_df = group_divs_by_year(df)
    
    div_consistacy_score = calculate_div_consistency_score(ann_df)
    
    div_growth_5y = get_div_growth_rate(ann_df,5)
    div_growth_10y = get_div_growth_rate(ann_df,10)
    div_growth_15y = get_div_growth_rate(ann_df,15)

    return div_consistacy_score, div_growth_5y, div_growth_10y, div_growth_15y

In [15]:
def get_all_div_analysis(stockList):
    for index, row in stockList.iterrows():
        try:
            stockObj = yf.Ticker(row['ticker'])
            
            div_analysis = get_div_analysis(stockObj)
            
            div_consis, div_growth_5y, div_growth_10y, div_growth_15y = get_div_analysis(stockObj)
            
        except:
            div_consis = 0.0
            div_growth_5y = 0.0
            div_growth_10y = 0.0
            div_growth_15y = 0.0
            
        stockList.at[index, 'div_consistency_score'] = div_consis
        stockList.at[index, 'div_growth_5y'] = div_growth_5y
        stockList.at[index, 'div_growth_10y'] = div_growth_10y
        stockList.at[index, 'div_growth_15y'] = div_growth_15y
        
    return stockList

In [16]:
def add_div_analysis(df):
    df['div_consistency_score'] = 0.0
    df['div_growth_5y'] = 0.0
    df['div_growth_10y'] = 0.0
    df['div_growth_15y'] = 0.0
    df = get_all_div_analysis(df)
    return df

In [17]:
def get_div_stocks(min_div = 3, min_consistency=.5):
    df = dividend_detective_scrape.get_data()
    
    df= df[df['div_yield']>min_div]
    
    df = add_div_analysis(df)   
    
    df = df[df['div_consistency_score']>min_consistency]
    df = df.sort_values('div_consistency_score', ascending=False)
    
    return df
    

In [18]:
df = get_div_stocks(min_div=13)

In [19]:
df

,ticker,name,annual_div,div_yield,div_consistency_score,div_growth_5y,div_growth_10y,div_growth_15y
531,USAC,USA Compression Partners,2.10,14.0,0.618595,0.002410,0.000000,0.0
438,SHLX,Shell Midstream Partners,1.84,15.9,0.612735,0.227597,0.000000,0.0
35,AM,Antero Midstream,1.23,13.6,0.589298,0.000000,0.000000,0.0
215,DHT,DHT Holdings,1.08,18.1,0.539641,1.254263,1.030651,0.0
258,EURN,Euronav,1.66,17.8,0.530847,2.395685,0.000000,0.0
